In [1]:
import requests

In [2]:
previous_api_url = 'https://scanr-api.enseignementsup-recherche.gouv.fr/api/v2/structures/search'
new_api_url = 'https://scanr.dataesr.ovh/api/scanr-organizations/_search'

In [31]:
user_input = 'Chimie Physique Microbiologie Environnement'

## 1. Previous query

In [32]:
body = {'query': user_input,
 'lang': 'fr',
 'page': '0',
 'pageSize': '20',
 'searchFields': ['label', 'alias'],
 'sourceFields': ['label', 'alias', 'id', 'address.city', 'kind'],
 'filters': {'kind': {'type': 'MultiValueSearchFilter',
   'op': 'any',
   'values': ['Structure de recherche', 'Secteur public']}}}

requests.post(previous_api_url, json=body).json()

{'request': {'query': 'Chimie Physique Microbiologie Environnement',
  'page': 0,
  'pageSize': 20,
  'lang': 'fr',
  'searchFields': ['label', 'alias'],
  'filters': {'kind': {'type': 'MultiValueSearchFilter',
    'op': 'any',
    'values': ['Structure de recherche', 'Secteur public']}},
  'sourceFields': ['label', 'alias', 'id', 'address.city', 'kind'],
  'aggregations': {}},
 'total': 0,
 'results': [],
 'facets': [{'id': 'facet_institutions', 'entries': []},
  {'id': 'facet_kind', 'entries': []},
  {'id': 'facet_badges', 'entries': []},
  {'id': 'facet_projects_types', 'entries': []},
  {'id': 'facet_urban_hits', 'entries': []},
  {'id': 'facet_natures', 'entries': []},
  {'id': 'facet_level', 'entries': []},
  {'id': 'facet_localisations', 'entries': []}]}

## 2. New query

In [33]:
# scanr web UI uses this list of fields to search in (the ^ gives a weight):
                        # ['id^5', 'acronym.*^5',
                        #   'externalIds.id^3',
                        #   'alias^3',
                        #   'label.*^3',
                        #   'description.*^2',
                        #   'address.city',
                        #   'address.address',
                        #   'address.postcode',
                        #   'publications.id^2',
                        #   'publications.title.default^2',
                        #   'publications.summary.default',
                        #   'publications.domains.label.*^2',
                        #   'publications.domains.code^2',
                        #   'projects.label.*^2',
                        #   'projects.acronym.*',
                        #   'projects.id^2',
                        #   'badges.label.*',
                        #   'badges.code',
                        #   'patents.id.keyword^2',
                        #   'patents.title.*^2',
                        #   'agreements.type^2',
                        #   'awards.label^2',
                        #   'awards.summary',
                        #   'web_content']

In [34]:
aggregations = {'byInstitutions': {'terms': {'field': 'institutions.label.keyword', 'size': 10}}}
# for facets, not mandatory

In [35]:
new_body = {
    'query': {
        'bool': {
            'must': [
                {
                    'query_string': {
                        'query': user_input,
                        "default_operator": "AND",
                        'fields': [ 'alias', 'label.*', 'acronym.*']

                    }
                }
            ],
            'filter': [
                {'term': {'isFrench': True}}, 
                {'terms': {'kind.keyword': ['Structure de recherche', 'Secteur public']}}
             ],
         },
 },
 'from': '0',
 'size': '20',
 '_source': ['id', 'alias', 'label', 'acronym', 'kind', 'address.city'],
 'aggs': aggregations
}


requests.post(new_api_url, json=new_body).json()

{'took': 45,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 22.426054,
  'hits': [{'_index': 'scanr-organizations-20240610',
    '_id': 'gfnuApABoKO6HjItH2M3',
    '_score': 22.426054,
    '_ignored': ['publications.title.fr.keyword',
     'publications.title.en.keyword',
     'publications.title.default.keyword'],
    '_source': {'id': '199412610M',
     'label': {'en': "Laboratoire de Chimie Physique et Microbiologie pour l'Environnement",
      'default': "Laboratoire de Chimie Physique et Microbiologie pour l'Environnement"},
     'acronym': {'en': 'LCPME', 'default': 'LCPME'},
     'kind': ['Secteur public', 'Structure de recherche'],
     'address': [{'city': 'Villers-lès-Nancy'}],
     'alias': ['UMR 7564',
      "Laboratoire de Chimie Physique et Microbiologie pour l'Environnement",
      'UMR7564',
      'UMR 7564']}}]},
 'aggregations': {'byInstitutions': {'doc_count_

In [10]:
new_body

{'query': {'bool': {'must': [{'query_string': {'query': 'colloide',
      'default_operator': 'AND',
      'fields': ['alias', 'label.*']}}],
   'filter': [{'term': {'isFrench': True}},
    {'terms': {'kind.keyword': ['Structure de recherche',
       'Secteur public']}}]}},
 'from': '0',
 'size': '20',
 '_source': ['id', 'alias', 'label', 'kind', 'address.city'],
 'aggs': {'byInstitutions': {'terms': {'field': 'institutions.label.keyword',
    'size': 10}}}}